# Setup

In [ ]:
# %pip install -r requirements.txt

# Environment Variables

In [ ]:
# Get environment variables

from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Google Colab Auth

In [ ]:
# Check if running in Google Colab

import sys

# If running in Colab, use the permissions of the currently authenticated user
if "google.colab" in sys.modules:
	print('Running in Google Colab')

	from google.colab import auth

	auth.authenticate_user()

# If not, set the GOOGLE_APPLICATION_CREDENTIALS to the service account credentials file
else:
	print("Running locally")

# Utilities


In [ ]:
# Import the utils module

from utils import *

# RAG Engine

In [ ]:
# https://cloud.google.com/vertex-ai/generative-ai/docs/rag-overview
# https://cloud.google.com/vertex-ai/generative-ai/docs/rag-quickstart

import os

import vertexai
from vertexai import rag
from vertexai.generative_models import GenerativeModel, Tool

PROJECT_ID = os.environ["PROJECT_ID"]
LOCATION = os.environ["REGION"]

# Initialize Vertex AI
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
# https://cloud.google.com/vertex-ai/generative-ai/docs/use-embedding-models

# Create RagCorpus
# Configure embedding model, for example "text-embedding-004".
embedding_model_config = rag.EmbeddingModelConfig(publisher_model="publishers/google/models/text-embedding-004")

backend_config = rag.RagVectorDbConfig(rag_embedding_model_config=embedding_model_config)

rag_corpus = rag.create_corpus(
    display_name="test3",
    backend_config=backend_config,
)

# List the rag corpus you just created
# rag_corpus = rag.list_corpora()

# Import Files to the RagCorpus
corpus_name = rag_corpus.name
print(corpus_name)

transformation_config = rag.TransformationConfig(
    chunking_config=rag.ChunkingConfig(
        chunk_size=512,
        chunk_overlap=100,
    ),
)

paths = ["gs://docs-10k/pdf"]

# Load from GCS bucket
GCS_BUCKET = "gs://docs-10k/pdf"
rag.import_files(
    corpus_name=corpus_name,
    paths=paths,
    max_embedding_requests_per_min=1000,  # Optional
    transformation_config=transformation_config, # Optional
)

# # Load from file
# rag_file_lyft = rag.upload_file(
#     corpus_name=rag_corpus.name,
#     display_name="lyft.pdf",
#     description="Lyft 10K for 2023",
#     path="docs/10k/pdf/lyft.pdf",
#     transformation_config=transformation_config,
# )
#
# rag_file_uber = rag.upload_file(
#     corpus_name=rag_corpus.name,
#     display_name="uber.pdf",
#     description="Uber 10K for 2023",
#     path="docs/10k/pdf/uber.pdf",
#     transformation_config=transformation_config,
# )

# Check that the corpus was created
print(rag.list_corpora())

# List the files in the rag corpus
print(rag.list_files(corpus_name))

# # Delete the corpus
# rag.delete_corpus(rag_corpus.name)

In [ ]:
# Direct context retrieval
rag_retrieval_config=rag.RagRetrievalConfig(
    top_k=3,  # Optional
    filter=rag.Filter(vector_distance_threshold=0.5)  # Optional
)

response = rag.retrieval_query(
    rag_resources=[
        rag.RagResource(
            rag_corpus=corpus_name,
            # Optional: supply IDs from `rag.list_files()`.
            # rag_file_ids=["rag-file-1", "rag-file-2", ...],
        )
    ],
    text="What was Uber's revenue in 2023?",
    rag_retrieval_config=rag_retrieval_config,
)
print(response)

In [ ]:
from IPython.display import Markdown

questions = [
    "What is the annual revenue of Uber?",
	"What is the annual revenue of Lyft?",
	"How does Uber's revenue compare to Lyft's revenue?",
	"Summarize Lyft's risk factors as bullet points?",
    "Who is the CEO of Uber and what was their largest source of revenue in 2023?",
]

# Enhance generation
# Create a RAG retrieval tool
rag_retrieval_tool = Tool.from_retrieval(
    retrieval=rag.Retrieval(
        source=rag.VertexRagStore(
            rag_resources=[
                rag.RagResource(
                    rag_corpus=corpus_name,  # Currently only 1 corpus is allowed.
                    # Optional: supply IDs from `rag.list_files()`.
                    # rag_file_ids=["rag-file-1", "rag-file-2", ...],
                )
            ],
            rag_retrieval_config=rag_retrieval_config,
        ),
    )
)
# Create a gemini model instance
rag_model = GenerativeModel(
    model_name="gemini-1.5-flash-001",
    tools=[rag_retrieval_tool]
)

# Generate response
for question in questions:
    print(question)
    response = rag_model.generate_content(question)
	#display(Markdown(response.text))
    print(response.text)
